In [46]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
from urllib.parse import urlparse
import re
import json
from tqdm import tqdm
import numpy as np

from requests_html import AsyncHTMLSession
import asyncio

#for dynamic web pages (Race Result Data)
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions

from multiprocessing import Pool, cpu_count 
import functools
import os


# Webscraping Ironman Data

**Author**: Migus Wong\
**Last Updated**: December 5, 2024

Website being scraped: https://www.coachcox.co.uk/imstats/im/



In [2]:
# Fetch the content of the webpage
url = 'https://www.coachcox.co.uk/imstats/im/'
response = requests.get(url)
webpage_content = response.content

# Parse the webpage content with Beautiful Soup
soup = BeautifulSoup(webpage_content, 'html.parser')


## Grabbing Series Data
Data on the website is broken down in terms of granularity as follows:

Series/Location -> Race -> Athlete

The webscraping strategy is as follows:
* Grab all data on each series.
* Within each series, scrape all races in that location.
* Within each race, scrape all results for each athlete.

**Note - World championship data was not scraped.*

This section below is how each race data

In [3]:
# Find all elements with the class 'ims-idx-racelist ims-collapse' which is broken down by continent
parent_elements = soup.find_all(class_='ims-idx-racelist ims-collapse') 

link_list = []
# Iterate over parent elements to find nested 'ims-idx-row-title' elements 
for parent in parent_elements: 
    #grab continent names
    continent_Name = parent.find(class_='head').get_text()
    continent_Name = continent_Name[:-1]
    rows = parent.find_all(class_='ims-idx-row') 
    for row in rows: 
        row_title = row.find(class_='ims-idx-row-title') 
        overview_link = row.find('a', class_='ims-idx-action flexible ims-red-button overview')
        region_name = row_title.get_text(strip=True)
        region_link = overview_link.get('href')
        parsed_url = urlparse(region_link)
        series_id = re.search(r'/series/(\d+)/',parsed_url.path).group(1)
        link_list.append([series_id,region_name,continent_Name,region_link])
        


In [4]:
series_df = pd.DataFrame(link_list,columns=['id','location','continent','link'])
series_df

,id,location,continent,link
0,332,Ironman Alaska,North America,https://www.coachcox.co.uk/imstats/series/332/
1,1,Ironman Arizona,North America,https://www.coachcox.co.uk/imstats/series/1/
2,37,Ironman Boulder,North America,https://www.coachcox.co.uk/imstats/series/37/
3,254,Ironman California,North America,https://www.coachcox.co.uk/imstats/series/254/
4,35,Ironman Canada,North America,https://www.coachcox.co.uk/imstats/series/35/
...,...,...,...,...
64,45,Ironman Taiwan,Asia Pacific,https://www.coachcox.co.uk/imstats/series/45/
65,28,Ironman Western Australia,Asia Pacific,https://www.coachcox.co.uk/imstats/series/28/
66,4,Ironman Brazil,South America,https://www.coachcox.co.uk/imstats/series/4/
67,41,Ironman Fortaleza,South America,https://www.coachcox.co.uk/imstats/series/41/


In [5]:
#save the following data to a csv
series_df.to_csv("./IronManData/sql/series.csv", index =False)

## Grabbing all Race Data

Now that we have the dataframe *series_df* which contains all of the series data, let's grab the individual race data.

In [6]:
links_list = series_df['link']

In [7]:
# Function to extract a single href attribute from HTML
def extract_href(html):
    if isinstance(html, str):  # Ensure the input is a string
        soup = BeautifulSoup(html, 'html.parser')
        link = soup.find('a', href=True)
        if link:
            return link['href']
    return "N/A"

In [8]:
#Function to grab race ID
def extract_race_ID(url):
    parsed_url = urlparse(url)
    race_id = re.search(r'/race/(\d+)/',parsed_url.path).group(1)
    return race_id

In [84]:
def scrape_series(url,regionID):
    response = requests.get(url)
    webpage_content = response.content
    soup = BeautifulSoup(webpage_content, 'html.parser')
    
    
    # Step 3: Find the script tag containing the desired JavaScript data
    script_tag = soup.find('script', string=re.compile('var imseriesindexdata ='))
    
    # Step 4: Extract the JavaScript data
    if script_tag:
        script_content = script_tag.string
        #print(script_content)
        # Use regular expressions to extract the JSON-like data
        pattern =r' \[(.*?)\]'
        data_match = re.search(pattern, script_content, re.DOTALL)
        if data_match:
            json_data = "[" +data_match.group(1) + "]"
            data = json.loads(json_data)
            df = pd.DataFrame(data)
    df.columns = ['year','link','konaSlots','male1st','female1st','finishers','dnf','dq']
    
    df['link'] =  df['link'].apply(extract_href)
    df['id'] = df['link'].apply(extract_race_ID)
    #Add series ID to df entries
    df['seriesID'] = regionID
    return df




In [85]:
#initialize empty dataframe to store all race data
races_df = pd.DataFrame()

#iterate through the series df and add all races to a single races df
for series in tqdm(series_df.itertuples(index=True, name="Pandas"), total=len(series_df)):
    series_single_df = scrape_series(series.link, series.id)
    races_df = pd.concat([races_df, series_single_df], ignore_index=True)

print("Race Scrape Successful!")

100%|██████████████████████████████████████████████████████████████████████████████████| 69/69 [00:42<00:00,  1.62it/s]

Race Scrape Successful!


In [86]:
races_df

,year,link,konaSlots,male1st,female1st,finishers,dnf,dq,id,seriesID
0,2022,https://www.coachcox.co.uk/imstats/race/1883/r...,45,33077,37383,718,125,,1883,332
1,2024,https://www.coachcox.co.uk/imstats/race/2166/r...,Male: 65<br />Female: 35,29466,35348,1440,191,2,2166,1
2,2023,https://www.coachcox.co.uk/imstats/race/2134/r...,Male: 20<br />Female: 60,29821,32654,1593,167,,2134,1
3,2022,https://www.coachcox.co.uk/imstats/race/1903/r...,55,27960,31358,1993,406,2,1903,1
4,2021,https://www.coachcox.co.uk/imstats/race/1768/r...,45,30210,35854,1932,358,9,1768,1
...,...,...,...,...,...,...,...,...,...,...
582,2011,https://www.coachcox.co.uk/imstats/race/49/res...,None,29136,30836,1505,228,,49,21
583,2010,https://www.coachcox.co.uk/imstats/race/48/res...,None,30208,33415,1548,106,,48,21
584,2009,https://www.coachcox.co.uk/imstats/race/47/res...,None,29852,33392,1306,214,,47,21
585,2008,https://www.coachcox.co.uk/imstats/race/46/res...,None,29903,34068,1403,164,,46,21


In [90]:
def extract_slots(text):
    slots = re.findall(r'\d+',str(text))
    if len(slots) == 1:
        if slots[0] == 0: #for instances where ther are 0 total slots
            return int(0),int(0),int(0)
        else: #for non-zero situations but do not know male-female breakdown
            return slots[0], np.nan, np.nan
    else: # if there is male female breakdown
        return int(int(slots[0])+int(slots[1])),slots[0],slots[1]

#We need to clean up some of the columns since the were listed as strings and not numerics
def process_df(race_df):
    cleaned_df = pd.DataFrame()
    # Only take the first 4 digits of each year (since there are 2 values with a month associated)
    cleaned_df['year'] = race_df['year'].astype(str).str.extract(r'(\d{4})').astype(int)
    cleaned_df['link'] = race_df['link']
    #let's split kona slots into 3 slots: total, male, female
    cleaned_df[['totalkonaSlots','maleKonaSlots','femaleKonaSlots']] = race_df['konaSlots'].fillna(0).apply(lambda x: pd.Series(extract_slots(x))) 
    cleaned_df['male1st'] = race_df['male1st'].fillna(0).astype(int)
    cleaned_df['female1st'] = race_df['female1st'].fillna(0).astype(int)
    cleaned_df['finishers'] = race_df['finishers'].fillna(0).astype(int)
    cleaned_df['dnf'] = race_df['dnf'].replace('',np.nan).fillna(0).astype(int)
    # Replace empty strings with NaN 
    cleaned_df['dq'] = race_df['dq']
    cleaned_df.loc[cleaned_df['dq'] == '', 'dq'] = np.nan
    cleaned_df['dq'] = pd.to_numeric(cleaned_df['dq'], errors='coerce')
    cleaned_df['dq'] = cleaned_df['dq'].fillna(0).astype(int)
    #finish copying over remaining df
    cleaned_df['id'] = race_df['id'].fillna(0).astype(int)
    cleaned_df['seriesID'] = race_df['seriesID'].fillna(0).astype(int)

    return cleaned_df

In [ ]:
races_df = process_df(races_df)

In [93]:
races_df.to_csv(f"./IronManData/sql/races.csv", index =False)

In [22]:
races_df['dnf']

0      125
1      191
2      167
3      406
4      358
      ... 
582    228
583    106
584    214
585    164
586    267
Name: dnf, Length: 587, dtype: int32

In [23]:
races_df.to_csv(f"./IronManData/sql/races.csv", index =False)

In [24]:
#determine most amount of DNFs
top_10 = races_df.nlargest(5, 'dnf')
top_10

,year,link,totalkonaSlots,maleKonaSlots,femaleKonaSlots,male1st,female1st,finishers,dnf,dq,id,regionid
17,2008,https://www.coachcox.co.uk/imstats/race/185/re...,0,NaN,NaN,30859,33289,1689,725,0,185,1
287,2019,https://www.coachcox.co.uk/imstats/race/479/re...,80,NaN,NaN,28562,33331,2024,700,22,479,11
385,2008,https://www.coachcox.co.uk/imstats/race/128/re...,0,NaN,NaN,29765,34212,1955,676,0,128,24
145,2008,https://www.coachcox.co.uk/imstats/race/139/re...,0,NaN,NaN,30838,35657,1793,659,0,139,15
215,2021,https://www.coachcox.co.uk/imstats/race/1871/r...,0,NaN,NaN,28156,30899,2259,623,17,1871,330


## Grabbing Race Result Data

We now have a comprehensive list of all Ironman Races that have occurred which are stored in the dataframe *races_df*. We now need to extrapolate the individual athlete results from each race.

In [25]:
def scrape_race_results(url,raceID):
    driver = webdriver.Chrome()
    driver.get(url)

    print(f"Processing race {raceID}")
    # Find all rows in the table
    rows = driver.find_elements(By.XPATH, '//table/tbody/tr')
    
    # Prepare to collect data
    all_rows_data = []
    
    # Iterate over each row of data with a progress bar
    for row in tqdm(rows, desc="Processing rows"):
        columns = row.find_elements(By.TAG_NAME, 'td')
        row_data = []
        # Iterate over each element
        for i, column in enumerate(columns):
            row_data.append(column.text)
            # Check the second (Athlete) and fifth (Division) columns for links
            if i in [1, 4]:
                links = column.find_elements(By.TAG_NAME, 'a')
                if links:
                    href = links[0].get_attribute('href')
                    row_data.append(href)
                else:
                    row_data.append(pd.NA)
        
        # Ensure the row has the expected number of columns before adding it to all_rows_data
        if len(row_data) >= 15:
            all_rows_data.append(row_data)
    
    driver.quit()
    
    # Convert the list of lists into a DataFrame
    df = pd.DataFrame(all_rows_data)
    df.columns = ['bib','Name','athleteLink','Country','Gender','Division','divLink','divisionRank','overallTime','overallRank','swimTime','swimRank','bikeTime','bikeRank','runTime','runRank','finishStatus','dnf']
    df['raceID'] = raceID
    return df


### The case for parallel processing

The code above works, but is *extremely* inefficient. Based off of testing results, because browsers are dynamically opened and iterated through, ~12 rows of data is processed each second. With 586 different races averaging around 2500 entries, processing time will take nearly ***34 hours***.

Below is my attempt at speeding up this process via parallel processing [here](https://medium.com/@fischiettigiulio/how-to-scrape-data-faster-with-selenium-and-parallel-scraping-advanced-98d4702f082). More details of the overall action plan can be found above, but the workflow goes as follow.

* master.py reads races file and divies up the records. Each subset of links and race IDs is associated a "thread". worker.py processes are spun up and kicked off.
* worker.py is what actually scrapes each web page.

In [26]:
#let's save the csv as a json file to make it slightly easier for the master script to work with.
r_path = "./IronManData/sql/races.csv"
w_path = "./IronManData/races.json"

df = pd.read_csv(r_path)
json_str = df.to_json(orient='records', lines=True)
with open(w_path,'w') as file:
    file.write(json_str)

This code block here can be used to run master.py

In [35]:
#!python master.py --num_workers 6

0 races already scraped. 587 races to scrape.
Exporting List to races_to_scrape.json
JSON file created successfully!


In [50]:
#After each individual csv data has been created, they need to be combined into a single csv as our "master" csv

# Directory containing the CSV files
directory = './IronManData/raceResultsData'

# Initialize an empty list to store individual DataFrames
data_frames = []

# Iterate through all CSV files in the directory
for filename in os.listdir(directory):
    if filename.endswith('.csv'):
        file_path = os.path.join(directory, filename)
        # Read the CSV file
        df = pd.read_csv(file_path)
        # Append the DataFrame to the list
        data_frames.append(df)

# Concatenate all DataFrames in the list
combined_df = pd.concat(data_frames, ignore_index=True)

# Write the combined DataFrame to a new CSV file
combined_df.to_csv('./IronManData/sql/results.csv', index=False)

print("All CSV files combined successfully!")


All CSV files combined successfully!


In [51]:
combined_df

,bib,Name,athleteLink,Country,Gender,Division,divLink,divisionRank,overallTime,overallRank,swimTime,swimRank,bikeTime,bikeRank,runTime,runRank,finishStatus,raceID,athleteID
0,40,Nils Frommhold,https://www.coachcox.co.uk/imstats/athlete/28963/,Germany,Male,MPRO,NaN,1.0,8:03:13,1.0,48:19,1.0,4:22:45,3.0,2:48:06,1.0,Finisher,1,28963.0
1,58,Paul Matthews,https://www.coachcox.co.uk/imstats/athlete/143...,United States,Male,MPRO,NaN,2.0,8:04:58,2.0,48:27,4.0,4:24:31,7.0,2:48:27,2.0,Finisher,1,143770.0
2,3,Tj Tollakson,https://www.coachcox.co.uk/imstats/athlete/33054/,United States,Male,MPRO,NaN,3.0,8:07:36,3.0,48:34,9.0,4:19:03,2.0,2:56:01,8.0,Finisher,1,33054.0
3,34,Tyler Butterfield,https://www.coachcox.co.uk/imstats/athlete/23792/,United States,Male,MPRO,NaN,4.0,8:14:41,4.0,52:28,22.0,4:26:02,10.0,2:52:02,4.0,Finisher,1,23792.0
4,41,Jarmo Hast,https://www.coachcox.co.uk/imstats/athlete/103...,Finland,Male,MPRO,NaN,5.0,8:16:09,5.0,51:36,17.0,4:32:03,16.0,2:48:36,3.0,Finisher,1,103580.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1096714,953,Maurus Oettl,https://www.coachcox.co.uk/imstats/athlete/238...,Germany,Male,M50-54,https://www.coachcox.co.uk/imstats/im/division...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,DNF,99,238791.0
1096715,184,Andreas Frischmann,https://www.coachcox.co.uk/imstats/athlete/237...,Germany,Male,M18-24,https://www.coachcox.co.uk/imstats/im/division...,NaN,NaN,NaN,1:00:53,263.0,NaN,NaN,NaN,NaN,DNF,99,237220.0
1096716,1981,Marco Meazza,https://www.coachcox.co.uk/imstats/athlete/238...,Italy,Male,M50-54,https://www.coachcox.co.uk/imstats/im/division...,NaN,NaN,NaN,NaN,NaN,5:26:08,638.0,NaN,NaN,DNF,99,238361.0
1096717,446,Roman Schaller,https://www.coachcox.co.uk/imstats/athlete/238...,Austria,Male,M18-24,https://www.coachcox.co.uk/imstats/im/division...,NaN,NaN,NaN,1:04:40,453.0,NaN,NaN,NaN,NaN,DNF,99,238204.0
